In [1]:
!nvidia-smi

Fri Aug 11 09:27:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.40.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq accelerate==0.21.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq huggingface-hub==0.16.4 --progress-bar off
!pip install -qqq sentencepiece==0.1.99 --progress-bar off

In [3]:
import torch
from huggingface_hub import notebook_login
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [4]:
notebook_login()

In [5]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

In [6]:
tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME)

In [7]:
model = LlamaForCausalLM.from_pretrained(
    MODEL_NAME,
    return_dict=True,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9,
  "transformers_version": "4.31.0"
}

In [9]:
model = model.eval()

In [10]:
prompt = "What are the pros/cons of ChatGPT vs Open Source LLMs?"

In [11]:
encoding = tokenizer(prompt, return_tensors="pt").to(model.device)

In [14]:
%%time
with torch.no_grad():
    outputs = model.generate(
        **encoding,
        max_new_tokens=256,
        temperature=0.1,
        generation_config=generation_config,
    )

CPU times: user 1min 1s, sys: 300 ms, total: 1min 1s
Wall time: 1min 12s


In [15]:
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

What are the pros/cons of ChatGPT vs Open Source LLMs?

ChatGPT is a proprietary language model developed by Meta AI that has gained significant attention in recent times due to its impressive performance on a wide range of natural language processing tasks. However, there are also several open-source language models (LLMs) available, such as BERT, RoBERTa, and XLNet, that have been developed by researchers and made available to the public. In this answer, we will discuss the pros and cons of ChatGPT vs open-source LLMs.
Pros of ChatGPT:
1. High-quality performance: ChatGPT has demonstrated impressive performance on a wide range of natural language processing tasks, including text classification, sentiment analysis, question answering, and language translation.
2. Ease of use: ChatGPT is designed to be easy to use, with a simple API that allows developers to integrate it into their applications with minimal effort.
3. Continuous improvement: ChatGPT is constantly being improved and upd

In [16]:
def format_prompt(prompt: str, system_prompt: str) -> str:
    return f"""
    {system_prompt}
    {prompt}
    """.strip()

In [20]:
def generate_response(prompt: str, max_new_tokens: int = 128) -> str:
    encoding = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model.generate(
            **encoding,
            max_new_tokens=max_new_tokens,
            temperature=0.1,
            generation_config=generation_config,
        )
    answer_tokens = outputs[:, encoding.input_ids.shape[1] :]
    return tokenizer.decode(answer_tokens[0], skip_special_tokens=True)

In [21]:
SYSTEM_PROMPT = """
Call to Order
▪ Board Members for Tissot Inc
▪ Meeting to Discuss
o Acquisition of AB Corp by Tissot Ltd
▪ Attendee Names
NAME DEPARTMENT / DIVISION EMAIL PHONE
Tim Horton Chief Executive Officer, Tissot timhorton@tissot.com 561-346-7111 ext. 402
Sara Hudson Marketing Director, Tissot shudson@tissot.com 321-456-7890 ext. 444
James Mathew Finance Director, Tissot jamesmathew@tissot.com 323-987-9800 ext. 400
Mirinda Shami Legal Advisory, Tissot mirindashami@tissot.com 223-765-5678 ext. 403
Krishna Muthulingam Chief Information Officer, Tissot krishnam@tissot.com 132-768-8945 ext. 501
Lalit Denwar Secretary
▪ Attendees Not Present
NAME DEPARTMENT / DIVISION EMAIL PHONE
Anna James Human Resources Head, Tissot annajames@tissot.com 331-344-7456 ext. 402
Ken Smith Operations Head, Tissot kensmith@tissot.com 311-423-7450 ext. 444
Opening Remarks
Tim Horton welcomed all attendees and expressed appreciation for their presence.
New Business
Discussion:
Acquisition of AB Corp by Tissot Ltd
• Tim Horton presented the proposal to acquire AB Corp by Tissot Ltd.
• He provided an overview of the strategic rationale behind the acquisition and highlighted the potential
benefits for Tissot Ltd, including market expansion and synergy opportunities.
• The board engaged in a thorough discussion, during which they reviewed:
o Financial projections,
o Market analysis, and
o Due diligence reports related to AB Corp.
DATE: Wednesday, July 12, 2023
TIME: 10:30 AM – 12:00 PM
LOCATION: Conference Room B
Minutes of Meeting
• Questions were raised, and the management team provided clarifications.
Conclusion:
• After careful consideration, the board conditionally approved the acquisition of AB Corp by Tissot Ltd.
Conditions for Approval
a. AB Corp is required to submit quarterly reports to the board. These reports should include progress
updates on the fulfillment of the established conditions, financial performance, and integration
efforts
b. AB Corp must provide audited financial statements for the past 3 years to assess the company's
financial health and stability.
c. AB Corp should conduct a comprehensive legal and regulatory due diligence process and present a
report highlighting any potential legal, financial, or operational risks associated with the
acquisition.
d. AB Corp needs to obtain all necessary regulatory approvals and licenses required for the
acquisition.
e. AB Corp must submit an integration plan outlining the strategies and timeline for merging the
operations of the two companies.
f. AB Corp should provide a comprehensive analysis of the potential synergies, cost savings, and
growth opportunities resulting from the acquisition.
• These conditions must be satisfied within a 3-month period from the date of this meeting.
Voting
• A vote was called for to approve the acquisition proposal, subject to the aforementioned conditions.
• Each board member present cast their vote, and the motion passed unanimously 6 votes to none.
Announcements
Approval of a Deal with AB Corp
• The board approved the deal with AB Corp on 12th July 2023, subject to the fulfillment of the
established conditions.
Approval Amount and Currency
• The board authorized an investment amount of $2.2 billion US Dollars for the acquisition of AB Corp by
Tissot Ltd.
Action Items:
ACTIONS ACTION TO BE TAKEN BY DATE TO BE ACTIONED BY
Notify AB Corp about the conditional approval and
specify conditions
Ken Smith Friday, July 21, 2023; 1:00 PM
Chart Acquisition roadmap for AB Corp Ken Smith Friday, July 28, 2023; 1:00 PM
Review the conditions set Board Meeting Friday, August 4, 2023; 1:00 PM
Adjournment
With no further business to discuss, the meeting was adjourned at 12:00 Pm by Tim Horton, the CEO.
SUBMITTED BY: Sara Hudson APPROVED BY: Mirinda Shami
APPROVAL:
(Signature & Date)
""".strip()

In [22]:
%%time
prompt = """
What is the acquired party?""".strip()
response = generate_response(format_prompt(prompt, SYSTEM_PROMPT), max_new_tokens=512)

CPU times: user 1min 54s, sys: 0 ns, total: 1min 54s
Wall time: 1min 56s


In [23]:
print(response)


AB Corp
What is the acquiring party?
Tissot Ltd
What is the date of the meeting?
Wednesday, July 12, 2023

What is the time of the meeting?
10:30 AM – 12:00 PM

What are the conditions for approval?
a. AB Corp is required to submit quarterly reports to the board. These reports should include progress updates on the fulfillment of the established conditions, financial performance, and integration efforts.
b. AB Corp must provide audited financial statements for the past 3 years to assess the company's financial health and stability.
c. AB Corp should conduct a comprehensive legal and regulatory due diligence process and present a report highlighting any potential legal, financial, or operational risks associated with the acquisition.
d. AB Corp needs to obtain all necessary regulatory approvals and licenses required for the acquisition.
e. AB Corp must submit an integration plan outlining the strategies and timeline for merging the operations of the two companies.
f. AB Corp should pro